In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), os.pardir))
from Tools.leica_tools import parse_lif
from Tools.exp_tools import Experiment

In [3]:
#parse_lif('/Volumes/T7/DMI/NKIP_FA_083/NKIP_FA_083.lif')

In [68]:
expID = 'NKIP_FA_084'
experiment = Experiment(expID)

In [77]:
experiment.generate_wp(labels=['Target', 'dead_Target', 'Effector', 'dead_Effector'], size=100)

2025-08-05 17:15:28.198789: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-05 17:15:28.318172: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-05 17:15:28.435355: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-05 17:15:28.551314: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-05 17:15:28.669636: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-05 17:15:28.787778: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-05 17:15:28.905561: W tensorflow/core/framework/local_rendezvous.cc:404] L

In [78]:
experiment.db.clear_annotations()

In [43]:
experiment.detect_droplets_legacy(mode='sweep')

In [52]:
df2 = pd.read_csv('/Users/fauberma/MPG/Experiments/NKIP_FA_084_alt/droplets.csv')

In [63]:
experiment.cell_count(model_name='cell_count_v3.h5')

2025-07-29 18:54:28.900581: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


3835/3835 ━━━━━━━━━━━━━━━━━━━━ 402s 105ms/step


2025-07-29 19:01:10.808777: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/fauberma/mambaforge/envs/phenotyping_v1/lib/python3.12/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


In [13]:
experiment.get_droplet_df()

,experiment_id,frame_id,x_min,y_min,x_max,y_max,outlier,time,condition
droplet_id,,,,,,,,,
0,NKIP_FA_084,0,5106,14745,5244,14883,False,1,Bulk
1,NKIP_FA_084,0,3751,3632,3889,3770,False,1,Bulk
2,NKIP_FA_084,0,5626,3548,5764,3686,False,1,Bulk
3,NKIP_FA_084,0,2857,3465,2995,3603,False,1,Bulk
4,NKIP_FA_084,0,5498,3455,5636,3593,False,1,Bulk
...,...,...,...,...,...,...,...,...,...
122693,NKIP_FA_084,31,5669,16058,5791,16180,False,8,Bulkstain
122694,NKIP_FA_084,31,3549,2905,3681,3037,False,8,Bulkstain
122695,NKIP_FA_084,31,97,464,227,594,False,8,Bulkstain


In [27]:
import pandas as pd

# Legacy transfer

In [3]:
import glob
import pandas as pd


def get_df(path):
    df = pd.read_csv(path)
    df.rename(
        columns={'GlobalID': 'droplet_id', 'Effector': 'effector', 'Target': 'target', 'dead_Effector': 'dead_effector',
                 'dead_Target': 'dead_target'}, inplace=True)
    df = df.astype({'droplet_id': int})
    df.set_index('droplet_id', inplace=True)
    df.drop(columns=['i'], inplace=True)
    return df

In [29]:
exp_id = 'NKIP_FA_082'
experiment = Experiment(exp_id)

In [30]:
# add droplets to db

droplets = pd.read_csv(os.path.join(os.getenv('EXP_DIR'), exp_id, 'droplets.csv'))
droplets.rename(columns={'GlobalID': 'droplet_id', 'frameID':'frame_id'}, inplace=True)
droplets['experiment_id'] = exp_id

experiment.db.add_droplets(droplets)
tfrecord_manifest = {
    'n_frames': droplets.index.size,
    'y_shape': 128,
    'x_shape': 128,
    'n_channels': experiment.handler.channel_df.index.size,
    'channel_info': experiment.handler.channel_df['channel_name'].to_dict(),
}
def frame_generator():
    for frame_id in experiment.frame_df.index:
        yield experiment.handler.get_frame(frame_id)
experiment.db.update_manifest(tfrecord_manifest)

In [31]:
experiment.db.clear_annotations()

In [32]:
# add cell_count predictions to annotations table
prefix = 'cell_count_v3'
droplets = pd.read_csv(os.path.join(os.getenv('EXP_DIR'), exp_id, 'droplets.csv'))
droplets.rename(columns={'GlobalID': 'droplet_id', 'frameID':'frame_id'}, inplace=True)
droplets['experiment_id'] = exp_id
droplets.set_index('droplet_id', inplace=True)
droplets.rename(columns={f'{prefix}_dead_Effector': 'dead_effector',
                         f'{prefix}_Target': 'target',
                         f'{prefix}_Effector': 'effector',
                         f'{prefix}_dead_Target': 'dead_target'}, inplace=True)
selected_annotations = ['dead_effector', 'dead_target', 'effector', 'target']
droplets[selected_annotations].head()

,dead_effector,dead_target,effector,target
droplet_id,,,,
0,0,0,1,0
1,0,0,0,1
2,0,0,0,2
3,0,0,0,0
4,0,0,1,0


In [33]:
source = 'cell_count_v3'
annotations = []

for droplet_id, droplet in droplets[selected_annotations].iterrows():
    for label_type, value  in droplet.items():
        annotations.append({'droplet_id': droplet_id, 'label_type': label_type, 'value': str(int(value)), 'source': source})

annotation_df = pd.DataFrame(annotations)
print(annotation_df.head())
experiment.db.add_annotations(annotation_df)

   droplet_id     label_type value         source
0           0  dead_effector     0  cell_count_v3
1           0    dead_target     0  cell_count_v3
2           0       effector     1  cell_count_v3
3           0         target     0  cell_count_v3
4           1  dead_effector     0  cell_count_v3


In [ ]:
# Import workpackages
wps = glob.glob(os.path.join(os.getenv('EXP_DIR'), exp_id, '*', 'WP_*.csv'))
annotations = []
for wp in wps:
    wp_id = os.path.basename(wp)
    df = get_df(wp).sort_values('droplet_id')

    for droplet_id, droplet in df.iterrows():
        for label_type, value in droplet.items():
            if not pd.isna(value):
                annotations.append({'droplet_id': droplet_id, 'label_type': label_type, 'value': str(int(value)), 'ap_id': wp_id})

annotations['status'] = 'completed'

annotation_df = pd.DataFrame(annotations)
experiment.db.add_annotations(annotation_df)